# **Python assignment**

In [3]:
# ---------------------------------------------------------
# File Name: MidTermAssignment.py
# Description: 
# Autor: Prat Paul; Gavini Charles; Fournier Justin; Blanc Mathieu
# Creation Date: 2024-10-16
# Version: 1.0
# ---------------------------------------------------------

Installing yahoofinance

In [2]:
#pip install yfinance

Note: you may need to restart the kernel to use updated packages.


Installing statsmodels

In [3]:
#pip install statsmodels

Note: you may need to restart the kernel to use updated packages.


In [4]:
#importations
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from statsmodels.graphics.tsaplots import plot_acf # import this function from this submodule
import statsmodels.api as sm
import scipy.stats as stats
from scipy.stats import gaussian_kde, norm, iqr, skew, kurtosis, jarque_bera, kstest, anderson
from statsmodels.stats.diagnostic import lilliefors
import scipy.signal as ss
import pylab

In [5]:
# Importing Nvidia stock from yahoo finance
Nvidia = yf.download("NVDA", start="1999-09-16", end="2024-09-16")
Nvidia

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
1999-09-16,0.051563,0.052083,0.050260,0.050911,0.046693,158112000
1999-09-17,0.050586,0.051042,0.048958,0.050781,0.046574,171648000
1999-09-20,0.049870,0.050521,0.048958,0.048958,0.044902,229104000
1999-09-21,0.047917,0.048177,0.043620,0.044271,0.040603,737328000
1999-09-22,0.044271,0.045573,0.041667,0.045313,0.041559,375984000
...,...,...,...,...,...,...
2024-09-09,104.879997,106.550003,103.690002,106.470001,106.460892,273912000
2024-09-10,107.809998,109.400002,104.949997,108.099998,108.090752,268283700
2024-09-11,109.389999,117.190002,107.419998,116.910004,116.900002,441422400
